## Examples on how to use premise's matrices with `bw_processing`

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook show exmaples on how to compute LCA scores using the matrices produced by `premise` via `.export_db_to_matrices()`, together with the matrix processing library `bw_processing` and `bw2calc`.

## Example of a regular LCA using premise's matrices

In [53]:
import bw2calc as bc
from bw2calc import __version__ as bw2calc_version
import bw_processing as bwp
from pathlib import Path
import csv
import numpy as np

In [54]:
# Set this to the folder containing A_matrix.csv and B_matrix.csv exports
# Example: Path("dev/export/remind/SSP2-NPi/2050")
fp_root = Path("path/to/export/folder")
fp_A_idx = fp_root / "A_matrix_index.csv"
fp_B_idx = fp_root / "B_matrix_index.csv"
fp_A_matrix = fp_root / "A_matrix.csv"
fp_B_matrix = fp_root / "B_matrix.csv"


In [55]:
def read_indices_csv(file_path: Path) -> dict:
    """
    Reads a CSV file and returns its contents as a dictionary.

    Each row of the CSV file is expected to contain four string values followed by an index.
    These are stored in the dictionary as a tuple of the four strings mapped to the index.

    """
    indices = dict()
    with open(file_path, encoding="utf-8") as read_obj:
        csv_reader = csv.reader(read_obj, delimiter=";")
        next(csv_reader, None)  # skip the headers
        for row in csv_reader:
            indices[(row[0], row[1], row[2], row[3])] = int(row[4])
    # remove any unicode characters
    indices = {tuple([str(x) for x in k]): v for k, v in indices.items()}
    return indices

A_idx = read_indices_csv(fp_A_idx)
B_idx = read_indices_csv(fp_B_idx)

In [56]:
def build_array(fp):
    array = np.genfromtxt(fp, delimiter=";", skip_header=1)
    
    # give `indices_array` a list of tuples of indices
    indices_array = np.array(
        list(zip(array[:, 1].astype(int), array[:, 0].astype(int))),
        dtype=bwp.INDICES_DTYPE,
    )
    
    data_array = array[:, 2]
    
    # make a boolean scalar array to store the sign of the data
    flip_array = array[:, -1].astype(bool)
    
    distributions_array = np.array(
        list(
            zip(
                array[:, 3].astype(int),  # uncertainty type
                array[:, 4].astype(float),  # loc
                array[:, 5].astype(float),  # scale
                array[:, 6].astype(float),  # shape
                array[:, 7].astype(float),  # minimum
                array[:, 8].astype(float),  # maximum
                array[:, 9].astype(bool),  # negative
            )
        ),
        dtype=bwp.UNCERTAINTY_DTYPE,
    )
    return indices_array, data_array, flip_array, distributions_array

In [57]:
# we create a static data package
dp = bwp.create_datapackage()

In [58]:
indices_array, data_array, flip_array, _ = build_array(fp_A_matrix)

# we add the technosphere vector
dp.add_persistent_vector(
    matrix="technosphere_matrix",
    indices_array=indices_array,
    data_array=data_array,
    flip_array=flip_array,
    #distributions_array=distributions_array,
)

indices_array, data_array, _, _ = build_array(fp_B_matrix)

# we add the biosphere vector
dp.add_persistent_vector(
    matrix="biosphere_matrix",
    indices_array=indices_array,
    data_array=data_array,
    flip_array=None,
    #distributions_array=distributions_array,
)

In [59]:
# let's build a characterization matrix for fossil CO2 = 1
# more complex LCIA vector building is possible 
# if one accesses bw2data.Method()
c_indices = np.array(
    [
        (v, v) 
        for k, v in B_idx.items() 
        if "carbon dioxide, fossil" in str(k).lower()
    ],
    dtype=bwp.INDICES_DTYPE   
)
c_data = np.ones(len(c_indices))

In [60]:
# we add the method vector
dp.add_persistent_vector(
    matrix='characterization_matrix',
    indices_array=c_indices,
    data_array=c_data,
)

In [61]:
# let's look for a gasoline car
ix = [
    v for k, v in A_idx.items() 
    if "transport, passenger car" in str(k)
    and "gasoline" in str(k)
][0]
ix

25553

In [62]:
lca = bc.LCA(
    demand={ix: 1},
    data_objs=[
        dp,
    ],
    use_distributions=False,
)

In [63]:
lca.lci()
lca.lcia()
lca.score

0.27376991977578063

## Scenarios

Now, let's vary the input of gasoline in the car dataset (and related CO2 emissions).

In [64]:
# let's build reverse indices' lists for convenience
rev_A_idx = {v: k for k, v in A_idx.items()}
rev_B_idx = {v: k for k, v in B_idx.items()}

In [65]:
# let's print the inputs labels and indices to the car dataset
for i in np.argwhere(lca.technosphere_matrix[:, ix]):
    i_ = lca.activity_dict.reversed[i[0]]
    label = rev_A_idx[i_]
    if label[0] == 'market for petrol, low-sulfur':
        gasoline_ix = i_
gasoline_values = np.arange(4/100, 10/100, 0.01)

print("Gasoline input index", gasoline_ix)
print("Scenario values", gasoline_values)

Gasoline input index 28947
Scenario values [0.04 0.05 0.06 0.07 0.08 0.09]


In [66]:
# current gasoline consumption
lca.technosphere_matrix[lca.dicts.product[gasoline_ix], lca.dicts.activity[ix]] * -1

0.08797021909304188

In [67]:
# same for CO2 emissions
# let's print the inputs labels and indices to the car dataset
for i in np.argwhere(lca.biosphere_matrix[:, ix]):
    i_ = lca.biosphere_dict.reversed[i[0]]
    label = rev_B_idx[i_]
    if label[0] == 'Carbon dioxide, fossil':
        co2_ix = i_
print(co2_ix)
co2_values = gasoline_values * 3.15 # kg CO2/kg gasoline
print(co2_values)

401
[0.126  0.1575 0.189  0.2205 0.252  0.2835]


In [68]:
# current CO2 emissions of the car
lca.biosphere_matrix[lca.biosphere_dict[co2_ix], lca.dicts.activity[ix]]

0.13075946643594696

In [69]:
# we create a second data package, to store our scenario values
dp_scenarios = bwp.create_datapackage(sequential=True)

In [70]:
dp_scenarios.add_persistent_array(
    matrix='technosphere_matrix',
    indices_array=np.array([(gasoline_ix, ix)], dtype=bwp.INDICES_DTYPE),
    data_array=np.array([tuple(gasoline_values)]).reshape(1, -1),
    flip_array=np.array([True])
)

In [71]:
dp_scenarios.add_persistent_array(
    matrix='biosphere_matrix',
    indices_array=np.array([(co2_ix, ix)], dtype=bwp.INDICES_DTYPE),
    data_array=np.array([tuple(co2_values)]).reshape(1, -1),
)

Now we can iterate over the LCA object (which is a generator) for as many times as we have scenario values.

In [72]:
for v, val in enumerate(gasoline_values):
    if v == 0:
        lca = bc.LCA(
            demand={ix: 1},
            data_objs=[
                dp, dp_scenarios
            ],
            use_distributions=False,
            use_arrays=True,
        )
        lca.lci()
        lca.lcia()
    else:
        next(lca)
    print(
        "Fuel:", -1 * lca.technosphere_matrix[lca.dicts.product[gasoline_ix], lca.dicts.activity[ix]], 
        "CO2:", lca.biosphere_matrix[lca.biosphere_dict[co2_ix], lca.dicts.activity[ix]], 
        "Score:", lca.score
    )

Fuel: 0.04 CO2: 0.126 Score: 0.2378711545922471
Fuel: 0.05 CO2: 0.1575 Score: 0.2758625359818526
Fuel: 0.060000000000000005 CO2: 0.189 Score: 0.31385391737145824
Fuel: 0.07 CO2: 0.2205 Score: 0.3518452987610632
Fuel: 0.08000000000000002 CO2: 0.25200000000000006 Score: 0.38983668015066975
Fuel: 0.09000000000000001 CO2: 0.28350000000000003 Score: 0.42782806154027464
